In [2]:
import os
import keras
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

from sklearn.model_selection import train_test_split

from tqdm import tqdm

from os import makedirs
from os.path import expanduser, exists, join

from keras.models import Model
from keras.layers import Input, Merge

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
batch_size = 16
img_width, img_height = 256, 256
num_classes = 2
n_epoches = 20
input_shape = (img_height, img_width, 3)

In [4]:
train_data_dir = "../data/train"
valid_data_dir = "../data/valid"

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                featurewise_center=False,  # set input mean to 0 over the dataset
                                samplewise_center=True,  # set each sample mean to 0
                                featurewise_std_normalization=False,  # divide inputs by std of the dataset
                                samplewise_std_normalization=True,  # divide each input by its std
#                                 zca_whitening=False,  # apply ZCA whitening
#                                 rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
                                width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                                height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                                horizontal_flip=True,  # randomly flip images
                                vertical_flip=True) # randomly flip images                                

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    seed=10)

val_datagen = ImageDataGenerator(rescale=1. / 255)

valid_generator = val_datagen.flow_from_directory(valid_data_dir,
                                                target_size=(img_height, img_width),
                                                batch_size=batch_size,
                                                seed=10)

print ("train data: ", train_generator.n)
print ("valid data: ", valid_generator.n)

Found 204 images belonging to 2 classes.
Found 46 images belonging to 2 classes.
('train data: ', 204)
('valid data: ', 46)


In [9]:
input  =  Input(shape=input_shape)
x = Conv2D(32, kernel_size=(3, 3),activation='relu', padding = 'same')(input)
x = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(x)
x = Conv2D(64, (3, 3), activation='relu', padding = 'same')(x)
feat1 = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(x)

x = Conv2D(128, kernel_size=(3, 3),activation='relu', padding = 'same')(feat1)
x = MaxPooling2D(pool_size=(2, 2))(x)
feat2 = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(x)

x = Conv2D(256, (3, 3), activation='relu', padding = 'same')(feat2)
x = MaxPooling2D(pool_size=(2, 2))(x)
feat3 = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(x)

added = keras.layers.add()(feat1, feat2, feat3)
z = GlobalAveragePooling2D(added)
z = Flatten()(z)
out = Dense(num_classes, activation='softmax')(z)

TypeError: add() takes exactly 1 argument (0 given)

In [9]:
base_model = Model([input, out])
base_model.summary()

ValueError: The first layer in a Sequential model must get an `input_shape` or `batch_input_shape` argument.

In [47]:
base_model.compile(Adam(lr=.001, decay=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
base_model.save_weights("jbm_custom_weights_15.h5")

In [48]:
base_model.fit_generator(mi,
                      steps_per_epoch = train_generator0.n // batch_size,
                      validation_data = valid_generator,
                      validation_steps = valid_generator.n // batch_size,
                      epochs = n_epoches,
                      verbose = 1)

Epoch 1/100


ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [ ]:
base_model.save_weights("jbm_inception_weights_5.h5")